In [1]:
import pandas as pd
import numpy as np

# Ustawienia wyświetlania
pd.set_option('display.max_rows', None)       # pokaż wszystkie wiersze
pd.set_option('display.max_columns', None)    # pokaż wszystkie kolumny
pd.set_option('display.width', None)          # brak ograniczenia szerokości
pd.set_option('display.max_colwidth', None)   # pełna szerokość kolumny (np. dla tekstu)


In [125]:
stock_list = pd.read_csv('stock_list.csv')
income = pd.read_csv('income.csv')
market_val = pd.read_csv('market_value.csv')
profitability = pd.read_csv('profitability.csv')
cash_flow = pd.read_csv('cash_flow.csv')
liabilities = pd.read_csv('liabilities.csv')
liquidity = pd.read_csv('liquidity.csv')
activity = pd.read_csv('activity.csv')

In [126]:
# Łączenie DataFrame'ów po kolei bez użycia reduce
all_data = income.merge(market_val, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(profitability, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(cash_flow, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liabilities, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liquidity, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(activity, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner')

# Podgląd wyniku
all_data.head()

,Nazwa,Ticker,Kwartały,Data,Przychody ze sprzedaży,Zysk ze sprzedaży,Zysk operacyjny (EBIT),Zysk z działalności gospodarczej,Zysk przed opodatkowaniem,Zysk netto,EBITDA,Kurs,Wartość księgowa na akcję,Cena/WK,Zysk na akcję,Cena/Zysk,Przychody ze sprzedaży na akcję,Cena/Przychód,Zysk operacyjny,Cena/Zysk operacyjny,Wartość księgowa Grahama na akcję,Cena/Wartość księgowa Grahama,Wartość przedsiębiorstwa na akcję,Wartość przedsiębiorstwa/Przychody,Wartość przedsiębiorstwa/EBITDA,Wartość przedsiębiorstwa/EBIT,ROE,ROA,Marża zysku operacyjnego,Marża zysku netto,Marża zysku ze sprzedaży,Marża zysku brutto,Marża zysku brutto ze sprzedaży,Rentowność operacyjna aktywów,Udział zysku netto w przepływach operacyjnych,Wskaźnik źródeł finansowania inwestycji,Zadłużenie ogólne,Zadłużenie kapitału własnego,Zadłużenie długoterminowe,Zadłużenie środków trwałych,Pokrycie aktywów trwałych kapitałami stałymi,Trwałość struktury finansowania,Zastosowanie kapitału obcego,Wskaźnik ogólnej sytuacji finansowej,Zadłużenie netto,Zadłużenie netto / EBITDA,Zadłużenie finansowe netto,Zadłużenie finansowe netto / EBITDA,I stopień pokrycia,II stopień pokrycia,Płynność gotówkowa,Płynność szybka,Płynność bieżąca,Płynność podwyższona,Pokrycie zobowiązań należnościami,Udział kapitału pracującego w aktywach,Pokrycie kosztów kapitałem obrotowym,Rotacja należności,Cykl należności,Cykl zobowiązań,Rotacja zapasów,Cykl zapasów,Rotacja majątku obrotowego,Rotacja majątku trwałego,Rotacja majątku ogółem,Cykl operacyjny,Cykl konwersji gotówki
0,06MAGNA,06N,2005/Q4,2006-02-14,-9 702,-283,-283,-2 457,-2 457,-785,NaN,"62,00","69,22","0,90","0,93","66,94","2,59","23,93","-0,30",NaN,"44,87","1,38","32,04","12,37","-118,18","-108,22","1,34%","0,89%","-11,43%","35,75%","-11,43%","-11,43%",NaN,"-0,28%","6,28%","3,70%","0,24","0,36",NaN,NaN,NaN,NaN,"0,35","0,71",-73 599 000,"90,31",-90 034 000,"110,47","2,01","2,01","1,65","2,77","2,82","1,99","0,09","0,43","15,68","0,42",868,2 920,"1,49",245,"0,04","0,07","0,03",1 113,-1 807
1,06MAGNA,06N,2006/Q1,2006-05-15,1 150,-667,-667,-667,-667,633,NaN,"69,00","69,50","0,99","0,86","79,89","-4,08",NaN,"-0,60",NaN,"45,11","1,53","43,16",NaN,"-51,48","-72,08","1,24%","0,85%","14,68%","-21,17%","14,68%","14,68%",NaN,"-0,59%","7,64%","11,65%","0,22","0,32",NaN,NaN,NaN,NaN,"0,33","0,67",-76 836 000,"30,49",-77 652 000,"30,81","2,01","2,01","1,60","2,93","3,00","2,13","0,13","0,44","-13,92","-0,74",-496,-1 975,"-1,76",-207,"-0,06","-0,11","-0,04",-703,1 272
2,06MAGNA,06N,2006/Q2,2006-08-14,2 799,1 539,1 539,1 539,1 539,3 707,NaN,"76,00","70,47","1,08","1,14","66,93","0,47","160,75","-0,45",NaN,"77,74","0,98","29,79","63,01","-39,60","-66,57","1,61%","1,06%","-94,65%","240,18%","-94,65%","-374,52%",NaN,"-0,42%","5,95%",NaN,"0,25","0,38",NaN,NaN,NaN,NaN,"0,25","5,79",-63 315 000,"28,00",-138 888 000,"61,43","22,72","22,72","1,77","3,85","3,93","1,79","0,11","0,72","63,46","0,12",3 118,17 994,"0,48",755,"0,01","0,02","0,00",3 873,-14 121
3,06MAGNA,06N,2006/Q3,2006-11-14,18 282,17 507,17 507,17 507,17 507,20 167,NaN,"88,00","31,50","2,79","8,03","10,96","4,17","21,11","6,02","14,62","46,12","1,91","79,93","19,18","13,31","13,28","25,49%","9,95%","144,43%","192,65%","144,43%","127,08%",NaN,"7,46%","39,45%",NaN,"0,35","0,88","0,05","4,04","4,93","0,41","0,38","1,77",46 372 000,"2,57",-24 249 000,"-1,34","4,71","4,93","0,38","2,71","2,81","0,47","0,10","0,59","-25,94","1,58",230,2 194,"-1,02",-358,"0,05","0,21","0,04",-128,-2 322
4,06MAGNA,06N,2006/Q4,2007-03-01,-16 331,-25 673,-25 695,-25 695,-25 695,2 235,NaN,"97,60","30,73","3,18","7,65","12,77","1,96","49,72","-2,43",NaN,"39,63","2,46","77,62","39,54","-32,04","-31,89","24,88%","12,09%","-124,00%","389,46%","-123,63%","-124,00%",NaN,"-3,85%","83,15%",NaN,"0,37","0,77",NaN,NaN,NaN,NaN,"0,37","1 111,18",7 704 000,"-1,06",-60 059 000,"8,25","2 979,26","2 979,26","0,85","2,68","2,68","0,89","0,00","0,63","9,05","0,95",385,4 607,"2,94",124,"0,03","0,18","0,02",509,-4 098


In [127]:
all_data['Ticker'].unique()

array(['06N', 'OCTAVA', '11-BIT-STUDIOS', 'ATAL', '3RG', '4MS', 'AAT',
       'ABE', 'ASSECO-BUSINESS-SOLUTIONS', 'ACG', 'ASSECO-POLAND',
       'ACTION', 'ADIUVO-INVESTMENT', 'AGORA', 'AGROTON', 'ALE', 'ALG',
       'ALTUS', 'ALL', 'ALIOR-BANK', 'AMBRA', 'AMICA', 'ANR',
       'APS-ENERGIA', 'APLISENS', 'APR', 'APATOR', 'ARH', 'ARTIFEX',
       'ASBISC', 'ASSECO-SOUTH-EASTERN-EUROPE', 'ASTARTA', 'ARCTIC-PAPER',
       'ATENDE', 'ATM-GRUPA', 'ATLANTA-POLAND', 'ATREM', 'ATS', 'ATT',
       'AIRWAY-MEDIX', 'B24', 'BBI-DEVELOPMENT', 'BBT', 'BETACOM', 'BCS',
       'BCX', 'BUDIMEX', 'BENEFIT-SYSTEMS', 'BANK-HANDLOWY', 'BIOTON',
       'BIO-PLANET', 'BLOOBER-TEAM', 'BUMECH', 'BIOMAXIMA', 'BNP',
       'BANK-OCHRONY-SRODOWISKA', 'BOWIM', 'BORYSZEW', 'BEST', 'CAP',
       'INTER-CARS', 'CAV', 'CBF', 'CCC', 'CDL', 'CD-PROJEKT', 'CEZ',
       'CI-GAMES', 'COLUMBUS-CAPITAL', 'CLOUD-TECHNOLOGIES',
       'COAL-ENERGY', 'CLNPHARMA', 'COMP', 'COGNOR', 'CAPITAL-PARTNERS',
       'CPD', 'COMPERIA-PL'

In [128]:
def clean_numeric_column(col):
    # Zamień wartości na stringi i oczyść z spacji oraz przecinków
    col_cleaned = col.astype(str).str.replace(' ', '', regex=False).str.replace(',', '.', regex=False).str.replace('%', '', regex=False)

    # Ustaw NaN dla pustych lub błędnych stringów
    col_cleaned = col_cleaned.where(~col_cleaned.isin(['', 'nan', 'NaN', 'None']), np.nan)

    # Spróbuj zamienić na liczby
    numeric_col = pd.to_numeric(col_cleaned, errors='coerce')

    # Jeśli większość wartości po konwersji to liczby (np. > 80%), przyjmujemy że kolumna jest liczbowa
    ratio_numeric = numeric_col.notna().sum() / len(numeric_col)
    if ratio_numeric > 0.4:
        return numeric_col  # konwersja udana
    else:
        return col  # zachowaj oryginalną kolumnę (tekstową)

# Przetwórz tylko kolumny typu object lub string
for col in all_data.select_dtypes(include=['object', 'string']).columns:
    all_data[col] = clean_numeric_column(all_data[col])

all_data['Data'] = pd.to_datetime(all_data['Data'], errors='coerce')

In [129]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20287 entries, 0 to 20286
Data columns (total 67 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   Nazwa                                          20287 non-null  object        
 1   Ticker                                         20287 non-null  object        
 2   Kwartały                                       20287 non-null  object        
 3   Data                                           20287 non-null  datetime64[ns]
 4   Przychody ze sprzedaży                         19447 non-null  float64       
 5   Zysk ze sprzedaży                              19447 non-null  float64       
 6   Zysk operacyjny (EBIT)                         19447 non-null  float64       
 7   Zysk z działalności gospodarczej               19447 non-null  float64       
 8   Zysk przed opodatkowaniem                      19447 non

In [130]:
all_data.to_csv('app_all_data.csv', index=False)

In [131]:
indicators = income[['Nazwa', 'Ticker', 'Kwartały', 'Data']].merge(market_val, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(profitability, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner') \
                  .merge(liabilities, on=['Nazwa', 'Ticker', 'Kwartały'], how='inner')

indicators['Data'] = pd.to_datetime(indicators['Data'], errors='coerce')

indicators_latest = indicators.loc[indicators.groupby('Ticker')['Data'].idxmax()].reset_index(drop=True)

for col in indicators_latest.select_dtypes(include=['object', 'string']).columns:
    indicators_latest[col] = clean_numeric_column(indicators_latest[col])

indicators_latest.drop(columns=['Kwartały', 'Data', 'Kurs'], inplace=True)

indicators_latest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 36 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Nazwa                                         351 non-null    object 
 1   Ticker                                        351 non-null    object 
 2   Wartość księgowa na akcję                     351 non-null    float64
 3   Cena/WK                                       331 non-null    float64
 4   Zysk na akcję                                 351 non-null    float64
 5   Cena/Zysk                                     235 non-null    float64
 6   Przychody ze sprzedaży na akcję               351 non-null    float64
 7   Cena/Przychód                                 338 non-null    float64
 8   Zysk operacyjny                               351 non-null    float64
 9   Cena/Zysk operacyjny                          241 non-null    flo

In [132]:
indicators_latest.to_csv('app_indicators.csv', index=False)

In [133]:
income_yearly = income[income['Kwartały'].str.contains('Q4', na=False)].copy()


for col in income_yearly.select_dtypes(include=['object', 'string']).columns:
    income_yearly[col] = clean_numeric_column(income_yearly[col])

income_yearly['Data'] = pd.to_datetime(income_yearly['Data'], errors='coerce')

income_yearly.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5251 entries, 3 to 21241
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Nazwa                             5251 non-null   object        
 1   Ticker                            5251 non-null   object        
 2   Kwartały                          5251 non-null   object        
 3   Data                              5251 non-null   datetime64[ns]
 4   Przychody ze sprzedaży            5038 non-null   float64       
 5   Zysk ze sprzedaży                 5038 non-null   float64       
 6   Zysk operacyjny (EBIT)            5038 non-null   float64       
 7   Zysk z działalności gospodarczej  5038 non-null   float64       
 8   Zysk przed opodatkowaniem         5038 non-null   float64       
 9   Zysk netto                        5251 non-null   float64       
 10  EBITDA                            0 non-null      fl

In [134]:
income_yearly.to_csv('app_income_yearly.csv', index=False)

In [42]:
base = pd.read_csv('Yearly_data/zysk.csv')

threshold = 0.8 * len(base)
base = base.loc[:, base.notnull().sum() >= threshold]
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6159 entries, 0 to 6158
Data columns (total 20 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Spółka                                             6159 non-null   object 
 1   Rok                                                6159 non-null   object 
 2   Przychody ze sprzedaży                             5910 non-null   float64
 3   Techniczny koszt wytworzenia produkcji sprzedanej  5910 non-null   float64
 4   Koszty sprzedaży                                   5904 non-null   float64
 5   Koszty ogólnego zarządu                            5905 non-null   float64
 6   Zysk ze sprzedaży                                  5913 non-null   float64
 7   Pozostałe przychody operacyjne                     6142 non-null   float64
 8   Pozostałe koszty operacyjne                        6138 non-null   float64
 9   Zysk ope

In [43]:
flow = pd.read_csv('Yearly_data/przeplywy.csv')

threshold = 0.8 * len(flow)
flow = flow.loc[:, flow.notnull().sum() >= threshold]
flow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6159 entries, 0 to 6158
Data columns (total 12 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   Spółka                                            6159 non-null   object 
 1   Rok                                               6159 non-null   object 
 2   Przepływy pieniężne z działalności operacyjnej    6113 non-null   float64
 3   Amortyzacja                                       6146 non-null   float64
 4   Przepływy pieniężne z działalności inwestycyjnej  6114 non-null   float64
 5   CAPEX (niematerialne i rzeczowe)                  6101 non-null   float64
 6   Przepływy pieniężne z działalności finansowej     6114 non-null   float64
 7   Emisja akcji                                      6098 non-null   float64
 8   Dywidenda                                         6101 non-null   float64
 9   Skup akcji         

In [44]:
bilans = pd.read_csv('Yearly_data/bilans.csv')

threshold = 0.8 * len(bilans)
bilans = bilans.loc[:, bilans.notnull().sum() >= threshold]
bilans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6159 entries, 0 to 6158
Data columns (total 32 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Spółka                                              6159 non-null   object 
 1   Rok                                                 6159 non-null   object 
 2   Aktywa trwałe                                       5919 non-null   float64
 3   Wartości niematerialne i prawne                     5916 non-null   float64
 4   Wartość firmy                                       5446 non-null   float64
 5   Rzeczowe składniki majątku trwałego                 5916 non-null   float64
 6   Należności długoterminowe                           5915 non-null   float64
 7   Inwestycje długoterminowe                           5915 non-null   float64
 8   Pozostałe aktywa trwałe                             5917 non-null   float64
 9

In [45]:
base = base[~base['Rok'].astype(str).str.contains('K|Q', regex=True, case=False)]
flow = flow[~flow['Rok'].astype(str).str.contains('K|Q', regex=True, case=False)]
bilans = bilans[~bilans['Rok'].astype(str).str.contains('K|Q', regex=True, case=False)]

base['Rok'] = pd.to_numeric(base['Rok'], errors='coerce')
flow['Rok'] = pd.to_numeric(flow['Rok'], errors='coerce')
bilans['Rok'] = pd.to_numeric(bilans['Rok'], errors='coerce')

In [46]:
df_merged = base.merge(flow, on=['Spółka', 'Rok'], how='outer') \
                .merge(bilans, on=['Spółka', 'Rok'], how='outer')

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5816 entries, 0 to 5815
Data columns (total 60 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Spółka                                              5816 non-null   object 
 1   Rok                                                 5816 non-null   int64  
 2   Przychody ze sprzedaży                              5579 non-null   float64
 3   Techniczny koszt wytworzenia produkcji sprzedanej   5579 non-null   float64
 4   Koszty sprzedaży                                    5573 non-null   float64
 5   Koszty ogólnego zarządu                             5574 non-null   float64
 6   Zysk ze sprzedaży                                   5582 non-null   float64
 7   Pozostałe przychody operacyjne                      5799 non-null   float64
 8   Pozostałe koszty operacyjne                         5795 non-null   float64
 9

In [47]:
significant_columns = ['Spółka', 'Rok', 'Przychody ze sprzedaży',  'Zysk netto', 'Zysk operacyjny (EBIT)', 'EBITDA', 
                       'Aktywa trwałe', 'Aktywa obrotowe', 'Aktywa razem', 'Kapitał własny akcjonariuszy jednostki dominującej', 
                       'Zobowiązania długoterminowe', 'Zobowiązania krótkoterminowe', 'Pasywa razem',
                       'Przepływy pieniężne z działalności operacyjnej', 'Przepływy pieniężne z działalności inwestycyjnej', 'Przepływy pieniężne razem']

financial_report = df_merged[significant_columns].copy()

financial_report.head()

,Spółka,Rok,Przychody ze sprzedaży,Zysk netto,Zysk operacyjny (EBIT),EBITDA,Aktywa trwałe,Aktywa obrotowe,Aktywa razem,Kapitał własny akcjonariuszy jednostki dominującej,Zobowiązania długoterminowe,Zobowiązania krótkoterminowe,Pasywa razem,Przepływy pieniężne z działalności operacyjnej,Przepływy pieniężne z działalności inwestycyjnej,Przepływy pieniężne razem
0,06N,2004,167714.0,12695.0,14756.0,18135.0,110071.0,190552.0,300623.0,204533.0,0.0,66891.0,300623.0,30735.0,15219.0,26450.0
1,06N,2005,7788.0,2315.0,-890.0,-815.0,103523.0,209064.0,312587.0,208041.0,0.0,74191.0,312587.0,44300.0,-1347.0,35055.0
2,06N,2006,5900.0,26742.0,-7316.0,-7282.0,31.0,189982.0,190013.0,92357.0,0.0,70858.0,190013.0,27635.0,42089.0,-62585.0
3,06N,2007,8313.0,-9512.0,2436.0,2449.0,16.0,45072.0,45088.0,8127.0,0.0,36961.0,45088.0,12495.0,30428.0,-54163.0
4,06N,2008,176739.0,8608.0,9818.0,16492.0,35019.0,64126.0,99145.0,38644.0,4308.0,56193.0,99145.0,15682.0,-8476.0,4976.0


In [48]:
financial_report['Lata raportu'] = financial_report.groupby('Spółka')['Rok'].transform(lambda x: x.max() - x)

financial_report.head()

,Spółka,Rok,Przychody ze sprzedaży,Zysk netto,Zysk operacyjny (EBIT),EBITDA,Aktywa trwałe,Aktywa obrotowe,Aktywa razem,Kapitał własny akcjonariuszy jednostki dominującej,Zobowiązania długoterminowe,Zobowiązania krótkoterminowe,Pasywa razem,Przepływy pieniężne z działalności operacyjnej,Przepływy pieniężne z działalności inwestycyjnej,Przepływy pieniężne razem,Lata raportu
0,06N,2004,167714.0,12695.0,14756.0,18135.0,110071.0,190552.0,300623.0,204533.0,0.0,66891.0,300623.0,30735.0,15219.0,26450.0,20
1,06N,2005,7788.0,2315.0,-890.0,-815.0,103523.0,209064.0,312587.0,208041.0,0.0,74191.0,312587.0,44300.0,-1347.0,35055.0,19
2,06N,2006,5900.0,26742.0,-7316.0,-7282.0,31.0,189982.0,190013.0,92357.0,0.0,70858.0,190013.0,27635.0,42089.0,-62585.0,18
3,06N,2007,8313.0,-9512.0,2436.0,2449.0,16.0,45072.0,45088.0,8127.0,0.0,36961.0,45088.0,12495.0,30428.0,-54163.0,17
4,06N,2008,176739.0,8608.0,9818.0,16492.0,35019.0,64126.0,99145.0,38644.0,4308.0,56193.0,99145.0,15682.0,-8476.0,4976.0,16


In [49]:
financial_report[financial_report['Spółka'] == '11-BIT-STUDIOS'].head(20)


,Spółka,Rok,Przychody ze sprzedaży,Zysk netto,Zysk operacyjny (EBIT),EBITDA,Aktywa trwałe,Aktywa obrotowe,Aktywa razem,Kapitał własny akcjonariuszy jednostki dominującej,Zobowiązania długoterminowe,Zobowiązania krótkoterminowe,Pasywa razem,Przepływy pieniężne z działalności operacyjnej,Przepływy pieniężne z działalności inwestycyjnej,Przepływy pieniężne razem,Lata raportu
20,11-BIT-STUDIOS,2009,0.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,0.0,0.0,100.0,0.0,0.0,100.0,15
21,11-BIT-STUDIOS,2010,474.0,-231.0,-284.0,-223.0,56.0,1296.0,1352.0,1268.0,0.0,63.0,1352.0,-705.0,-63.0,631.0,14
22,11-BIT-STUDIOS,2011,3055.0,1293.0,1459.0,1516.0,87.0,2899.0,2986.0,2561.0,0.0,306.0,2986.0,351.0,-73.0,567.0,13
23,11-BIT-STUDIOS,2012,4795.0,1345.0,1795.0,1913.0,96.0,7241.0,7337.0,6914.0,0.0,238.0,7337.0,-713.0,-129.0,2166.0,12
24,11-BIT-STUDIOS,2013,5709.0,1008.0,1394.0,1590.0,590.0,8123.0,8713.0,7922.0,0.0,284.0,8713.0,838.0,-648.0,276.0,11
25,11-BIT-STUDIOS,2014,17679.0,9236.0,11245.0,11532.0,1745.0,19770.0,21515.0,17005.0,0.0,2128.0,21515.0,3791.0,-1131.0,3903.0,10
26,11-BIT-STUDIOS,2015,23170.0,11566.0,13762.0,16231.0,7068.0,25750.0,32818.0,28676.0,571.0,3571.0,32818.0,18679.0,-2891.0,16116.0,9
27,11-BIT-STUDIOS,2016,27016.0,12929.0,14134.0,16703.0,9490.0,36159.0,45649.0,42126.0,444.0,3079.0,45649.0,12661.0,-5408.0,6657.0,8
28,11-BIT-STUDIOS,2017,19186.0,3558.0,7524.0,9238.0,17174.0,31476.0,48650.0,46223.0,763.0,1664.0,48650.0,8794.0,-10034.0,-211.0,7
29,11-BIT-STUDIOS,2018,82114.0,37550.0,46949.0,51442.0,41317.0,67946.0,109263.0,89912.0,12018.0,7333.0,109263.0,39046.0,-55572.0,-3926.0,6


In [73]:
# Załóżmy, że masz już df financial_report oraz parametr window

window = 3  # lub inna wartość

# Kolumny do analizy (bez 'Spółka', 'Rok', 'Lata raportu')
cols_to_analyze = [col for col in financial_report.columns if col not in ['Spółka', 'Rok', 'Lata raportu']]

# Wynikowy DataFrame
results = []

for company, group in financial_report.groupby('Spółka'):
    row = {'Spółka': company}
    for col in cols_to_analyze:
        try:
            val_0 = group.loc[group['Lata raportu'] == 0, col].values[0]
            val_w = group.loc[group['Lata raportu'] == window, col].values[0]
            if pd.notnull(val_0) and pd.notnull(val_w) and val_w != 0 and (val_0 / val_w) > 0:
                row[col] = round(((val_0 / val_w) ** (1/window) - 1) * 100, 2)
            else:
                row[col] = np.nan
        except IndexError:
            row[col] = np.nan
    results.append(row)

df_window_ratio = pd.DataFrame(results)
df_window_ratio.head(20)

,Spółka,Przychody ze sprzedaży,Zysk netto,Zysk operacyjny (EBIT),EBITDA,Aktywa trwałe,Aktywa obrotowe,Aktywa razem,Kapitał własny akcjonariuszy jednostki dominującej,Zobowiązania długoterminowe,Zobowiązania krótkoterminowe,Pasywa razem,Przepływy pieniężne z działalności operacyjnej,Przepływy pieniężne z działalności inwestycyjnej,Przepływy pieniężne razem
0,06N,47.71,38.08,-19.05,27.52,26.97,59.74,42.69,50.94,45.62,4.58,42.69,-74.73,262.13,163.08
1,11-BIT-STUDIOS,26.08,-37.84,NaN,-24.18,23.22,-10.41,6.12,5.23,-20.17,28.15,6.12,23.38,6.65,124.92
2,3RG,36.25,-11.85,-12.60,-11.75,0.00,20.72,2.29,8.98,NaN,-6.94,2.29,-46.95,NaN,NaN
3,4MS,49.80,105.64,94.02,85.12,22.03,49.07,36.78,42.05,-10.99,59.15,36.78,54.20,-37.63,-12.94
4,AAT,-13.79,3.77,6.37,5.97,9.73,-49.28,2.43,3.47,-61.68,-36.22,2.43,NaN,NaN,NaN
5,ABE,1.47,4.95,10.38,10.08,-1.86,4.94,4.23,6.77,0.65,3.10,4.23,NaN,-25.45,NaN
6,ACG,5.19,5.38,2.72,2.14,-3.87,6.33,1.36,2.50,-12.40,-0.48,1.36,14.49,-23.34,-14.80
7,ACTION,3.18,-25.80,-26.39,-23.66,-2.04,1.91,1.00,0.68,-16.64,9.71,1.00,NaN,NaN,-19.03
8,ADIUVO-INVESTMENT,-40.73,-37.69,NaN,NaN,-90.46,-46.39,-78.08,NaN,-59.06,-6.40,-78.08,-16.06,NaN,-41.15
9,AGORA,15.30,NaN,NaN,29.65,0.31,7.13,1.56,-3.66,3.13,7.22,1.56,27.04,28.11,NaN


In [55]:
# Calculating Zysk operacyjny (EBIT) for 11-BIT-STUDIOS with values: val_0=-2838.0, val_w=30031.0, window=3 and result=nan

((-2838.0 / 30031.0) ** (1/3) - 1)  * 100

(-77.22539412472045+39.44677449834083j)

In [66]:
# Załóżmy, że masz już df financial_report oraz parametr window

window = 3  # lub inna wartość

# Kolumny do analizy (bez 'Spółka', 'Rok', 'Lata raportu')
cols_to_analyze = ['Zysk operacyjny (EBIT)']

# Wynikowy DataFrame
results = []

for company, group in financial_report.groupby('Spółka'):
    row = {'Spółka': company}
    if company != '11-BIT-STUDIOS':
        continue
    for col in cols_to_analyze:
        try:
            val_0 = group.loc[group['Lata raportu'] == 0, col].values[0]
            val_w = group.loc[group['Lata raportu'] == window, col].values[0]
            if pd.notnull(val_0) and pd.notnull(val_w) and val_w != 0:
                row[col] = ((float(val_0) / float(val_w)) ** (1/window) - 1)  * 100
                if company == '11-BIT-STUDIOS':
                    print(f"Calculating {col} for {company} with values: val_0={val_0}, val_w={val_w}, window={window} and result={row[col]}")
            else:
                if company == '11-BIT-STUDIOS':
                    print(f"Skipping {col} for {company} due to invalid values: val_0={val_0}, val_w={val_w}")
                row[col] = np.nan
        except IndexError:
            if company == '11-BIT-STUDIOS':
                    print(f"IndexError for {col} in {company}, setting to NaN")
            row[col] = np.nan
    results.append(row)

df_window_ratio = pd.DataFrame(results)
df_window_ratio.head(20)

Calculating Zysk operacyjny (EBIT) for 11-BIT-STUDIOS with values: val_0=-2838.0, val_w=30031.0, window=3 and result=(-77.22539412472045+39.44677449834083j)


,Spółka,Zysk operacyjny (EBIT)
0,11-BIT-STUDIOS,-77.225394+39.446774j


In [78]:
financial_report.to_csv('Yearly_data/financial_report.csv', index=False)